Here, I try to solve the system of equations

\begin{align}
\frac{\partial c}{\partial t} 
&= \varepsilon\frac{\partial}{\partial x} \left(\frac{\partial c}{\partial x} + \rho\frac{\partial \phi}{\partial x}\right)\\
\frac{\partial \rho}{\partial t} 
&=  \varepsilon\frac{\partial}{\partial x} \left(\frac{\partial\rho}{\partial x}  + c \frac{\partial \phi}{\partial x}\right)\\
-\varepsilon^2 \frac{\partial^2\phi}{\partial x^2} &= \rho\\
\end{align}

with the boundary conditions

\begin{equation}
\begin{split}
\frac{\partial c}{\partial x} + \rho\frac{\partial \phi}{\partial x} &= 0
\quad\text{at $ x = 0$ or $x = 2$}\\
\frac{\partial \rho}{\partial x} + c\frac{\partial \phi}{\partial x} &= 0
\quad\text{at $ x = 0$ or $x = 2$}\\
\end{split}
\end{equation}

\begin{equation}
\begin{split}
\phi -\varepsilon\delta \frac{\partial \phi}{\partial x} &= -v \quad\text{at $x = 0$}\\
\phi+ \varepsilon\delta \frac{\partial \phi}{\partial x} &= v \quad\text{at $x = 2$}
\end{split}
\end{equation}

The initial values are
\begin{equation}
\begin{split}
c(x,0) &= 1\\
\rho(x,0) &= 0\\
\phi(x,0) &= vx\\
\end{split}
\end{equation}


In the nabla notation, the above Robin boundary condition for $ \phi $ is written as  

\begin{equation}
\hat{n}\cdot(\hat{a}\phi + b \nabla \phi) = v(\vec{x}),\quad\text{on $S_R$}
\end{equation}


where $\hat{a}= \hat{n}$   and   b=epsilon*delta

In [1]:
import fipy as fp
from matplotlib import pyplot as plt
import numpy as np

m = fp.Grid1D(nx=50,Lx=2)
v = 0.1
epsilon= 0.1
delta = 0.1
x = m.cellCenters[0]
xc=np.asarray(m.cellCenters)[0]

In [ ]:
##Initial conditions
c = fp.CellVariable(name="c", mesh=m , value=1., hasOld=True)

rho = fp.CellVariable(name="rho", mesh=m , value=0., hasOld=True)

phi = fp.CellVariable(name="phi",mesh=m, hasOld=True)
phi.setValue(v*(x-1))

#Plot the initial values
plt.plot(xc, phi.value, label='$\phi$')
plt.plot(xc, c.value, label='$c$')
plt.plot(xc, rho.value, label='$\\rho$')
plt.xlabel('x')
plt.legend(loc=7)
plt.show()

cFace = c.faceValue
rhoFace = rho.faceValue
phiFace = phi.faceValue

In [ ]:
#Calculate cell distance
from fipy.tools import numerix
MA = numerix.MA

tmp = MA.repeat(m._faceCenters[..., numerix.NewAxis,:], 2, 1)
cellToFaceDistanceVectors = tmp - numerix.take(m._cellCenters, m.faceCellIDs, axis=1)

tmp = numerix.take(m._cellCenters, m.faceCellIDs, axis=1)
tmp = tmp[..., 1,:] - tmp[..., 0,:]
cellDistanceVectors = MA.filled(MA.where(MA.getmaskarray(tmp), cellToFaceDistanceVectors[:, 0], tmp))

In [ ]:
mask = m.exteriorFaces
Gamma = fp.FaceVariable(mesh=m, value=epsilon*epsilon)
Gamma.setValue(0., where=mask)
dPf = fp.FaceVariable(mesh=m,
                   value=m._faceToCellDistanceRatio * cellDistanceVectors)
n = m.faceNormals
a = m.faceNormals
b = fp.FaceVariable(mesh=m, value=epsilon * delta, rank=0)
g = fp.FaceVariable(mesh=m, value=v, rank=0)
RobinCoeff = mask * epsilon * epsilon * n / (-dPf.dot(a) + b)


I understand that you are tryin to map

\begin{equation}
\begin{split}
\int_V\nabla\cdot\left(\nabla\phi\right) d V &= \int_S\hat{n}\cdot\nabla\phi d S\\
&\approx \sum_f\left(\hat{n}\cdot\nabla\phi\right)_f A_f\\
&=\sum_{f\notin S_R}(\hat{n}\cdot\nabla\phi) A_f 
+ \sum_{f\sum S_R} \left(\hat{n}\cdot\nabla\phi\right)_f A_f\\
&\approx \sum_{f\notin S_R}(\hat{n}\cdot\nabla\phi) A_f 
+\sum_{f\in S_R}\frac{v_f - (\hat{n}\cdot\hat{a})_f\phi_P}{-\left(\vec{d}_{fP}\cdot\vec{a}\right)_f + b_f}A_f
\end{split}
\end{equation}

into the python notebook. 

In the above code, the value of g is set to v. 
However, v = 1 throughout this workbook. The Robin boundary condition that I intended was

\begin{equation}
\begin{split}
\phi -\varepsilon\delta \frac{\partial \phi}{\partial x} &= -v \quad\text{at $x = 0$}\\
\phi+ \varepsilon\delta \frac{\partial \phi}{\partial x} &= v \quad\text{at $x = 2$}
\end{split}
\end{equation}

**Question**: As you can see, shouldn't the value of g be set to -v on the left and +v on the right? The present code gives the solution of  phi that is not symmetric. I expect phi to have some symmetric behavior near the left and right boundary.

In [ ]:
#Equation
Drho = epsilon*rho
Dc   = epsilon*c

eq1 = (fp.TransientTerm(var=c) == fp.DiffusionTerm(coeff=epsilon, var=c) +fp.DiffusionTerm(coeff=Drho, var=phi))
eq2 = (fp.TransientTerm(var=rho) == fp.DiffusionTerm(coeff=epsilon, var=rho) +fp.DiffusionTerm(coeff=Dc, var=phi))
eq3 = ((fp.DiffusionTerm(coeff=Gamma, var=phi) + (RobinCoeff * g).divergence
       - fp.ImplicitSourceTerm(coeff=(RobinCoeff).divergence, var=phi) + fp.ImplicitSourceTerm(coeff=1.0, var=rho)) == 0)
eqns = eq1 & eq2 & eq3 

Below, I have tried to implement the Newton method for the set of equations

\begin{equation}
\vec{F}(\vec{x}) = \left\{\begin{array}{l}
\displaystyle{ -\frac{\partial c}{\partial t} +\varepsilon\nabla\cdot\left(\nabla c+ \rho\nabla \phi \right)}\\
\,\\
\displaystyle{}
\displaystyle{-\frac{\partial \rho}{\partial t} +\varepsilon\nabla\cdot\left(\nabla \rho  + c \nabla \phi \right)}\\
\,\\
\displaystyle{\varepsilon^2 \nabla\cdot(\nabla \phi)+ \rho}
\end{array}
\right.
\end{equation}

The variaton in x leads to 

\begin{equation}
\vec{F}(\vec{x} + \delta\vec{x}) = \vec{F}(\vec{x}) + \left.\delta\vec{F}\right|_{\vec{x}}\approx 0
\end{equation}

where 


\begin{equation}
\delta \vec{F}(\vec{x}) \equiv 
\left\{
\begin{array}{ll}
\displaystyle{-\frac{\partial \delta c}{\partial t}+\varepsilon\nabla\cdot\nabla \delta c 
+ \varepsilon\nabla\cdot(\nabla\phi)\delta\rho 
+ \varepsilon\nabla\cdot\left(\rho\nabla\delta\phi\right)}\\
\,\\
\displaystyle{-\frac{\partial \delta \rho}{\partial t}+\varepsilon\nabla\cdot(\nabla\phi)\delta c
+\varepsilon\nabla\cdot\nabla\delta\rho
+ \varepsilon\nabla\cdot\left(c\nabla\delta\phi\right)}\\
\,\\
\displaystyle{\delta\rho 
+ \epsilon^2\nabla\cdot\left(\nabla\delta\phi\right)}
\end{array}
\right.
\end{equation}


The impletation is shown below:

In [ ]:
dc = fp.CellVariable(mesh=m, name=r"$\delta c$", hasOld=True)
drho = fp.CellVariable(mesh=m, name=r"$\delta rho$", hasOld=True)
dphi = fp.CellVariable(mesh=m, name=r"$\delta phi$", hasOld=True)

dcEq =((fp.TransientTerm(var=dc)==fp.DiffusionTerm(coeff=epsilon, var=dc)+fp.ConvectionTerm(coeff=epsilon*phi.faceGrad,var=drho)+fp.DiffusionTerm(coeff=epsilon*rho, var=dphi))+fp.ResidualTerm(equation=eq1))

drhoEq = ((fp.TransientTerm(var=drho)==fp.ConvectionTerm(coeff=epsilon*phi.faceGrad, var=dc)+fp.DiffusionTerm(coeff=epsilon,var=drho)+fp.DiffusionTerm(coeff=epsilon*c, var=dphi))+fp.ResidualTerm(equation=eq2))

dphiEq =((fp.ImplicitSourceTerm(coeff=1.0, var=drho)+fp.DiffusionTerm(coeff=epsilon**2, var=dphi))+fp.ResidualTerm(equation=eq3)) 

dEq = dcEq & drhoEq & dphiEq

Given the constraints below, the sweep routine seems to be functioning and the residue tends to go down. However, the solution was not right. I suspect that the Robin boudnary condition for dphi is not correct. In fact, all boundary conditions for the variational terms may not be correct. 

**Questions**: How do we treat the boundary condition on the varational terms?  Especially, the Robin boundary conditions on this term.

If there is anything else that are wrong in this notebook, please let me know. 

In [1]:
dc.constrain(0., where=m.facesRight)
dc.constrain(0., where=m.facesLeft)
drho.constrain(0., where=m.facesRight)
drho.constrain(0., where=m.facesLeft)
dphi.constrain(0, where=m.facesLeft)
#dphi.faceGrad.constrain([-phi.faceValue/(epsilon*delta)], where=m.facesRight)

NameError: name 'dc' is not defined

In [ ]:
dc.value = 0.
drho.value = 0.
dphi.value  = 0.

In [ ]:
##Sweep
newton = []
c.updateOld()
rho.updateOld()
phi.updateOld()
dc.updateOld()
drho.updateOld()
dphi.updateOld()
for sweep in xrange(10):
    res = dEq.sweep(dt=1000)
    c.value = c.value + dc.value
    rho.value = rho.value + drho.value
    phi.value = phi.value+drho.value
    newton.append([sweep, res, max(abs(dc)), max(abs(drho)), max(abs(dphi))])
    
newton = numerix.array(newton)
print(newton)

In [ ]:
##Print
plt.semilogy(newton[...,0], newton[..., 1], label="newton")
plt.ylabel("residual")
plt.xlabel("sweep")
plt.legend()
plt.show()

print(phi.value)
#Plot the results
plt.plot(xc, phi.value, label='$\phi$ - soln')
plt.plot(xc, c.value, label='$c$-soln')
plt.plot(xc, rho.value, label='$\\rho$ - soln')
plt.xlabel('x')
plt.legend(loc=7)
plt.show()